# Wikipedia Analysis

### Project 3 - Massive computing

By  Mireia Kesti (100406960) and Aleksandra Jamróz (100491363)

### Context - page rank algorithm
The goal of this algorithm was to set a weight for each webpage on the internet, based on the number of external links pointing to the page. The intention behind this was to objectively determine the interest of a webpage. 
The PageRank algorithm was proposed in 1998 by Larry Page and Sergey Brin and served as the foundation for the original Google search engine.

### Context - goals of this project
We will be implementing the original algorithm to set the weights for each page in Wikipedia (English version), using the image stored in the Databricks Databases Set. This database contains 5823210 entries, stored in a parquet set of files. For simplicity and optimization purposes we will not be using the complete dataset for the analysis of the structure, but a smaller version instead. 
To achieve this we will be using Apache Spark Dataframes to handle both the Wikipedia database and the intermediate results. The end-goal is to obtain a Pandas DF with three columns, namely the title of the page, its ID, and its pagerank. 


#### Steps we will be following
- 1. Configuration of the library
- 2. Checking of the raw data structure
- 3. Extraction of relevant data
- 4. Creation of a table of forward links
- 5. Computation of the number of outgoing links
- 6. Creation of a table of reverse links 
- 7. Creation of the page rank table 
- 8. Definition of the number of iterations for the page rank

# 1. Library configuration

In [0]:
import pandas as pd
import re
import math 
import numpy as np 

In [0]:
from pyspark.sql.types import * # Define the data types in the PySpark data model that will be used. Once the type of data is defined, it makes the analysis of data easier
from pyspark.sql.types import ArrayType, StringType, LongType, FloatType

from pyspark.sql.functions import *
from pyspark.sql.functions import size, explode, collect_list 

from pyspark.sql import functions as F # List of built-in functions available for DataFrame. Import them as F to avoid namespace coverage (such as pyspark sum function covering python built-in sum function)
from pyspark.sql import SparkSession # So we can access PySpark/Spark SQL capabilities in PySpark

from operator import add 



In [0]:
# To optimize when converting a Spark DataFrame to a Pandas DataFrame using the call toPandas() & when creating a Spark DataFrame from a Pandas DataFrame with createDataFrame(pandas_df), set to "true"
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# 2. Checking of the raw data structure
Now that our libraries are configured, we will import the dataset and inspect it in its raw form in order to get an idea of the data that we will be handling.

In [0]:
# The URI for the database, which contains 5823210 entries, stored in a parquet set of files.

# Create a Spark DataFrame from those parquet files

wikipediaDF = spark.read.parquet("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet")

As we can see, this database contains 7 columns with the following names and datatypes:
- title: string
- id: integer
- revisionId: integer
- revisionTimestamp: timestamp
- revisionUsername: string
- revisionUsernameId: integer
- text: string

The relevant information for our Wikipedia analysis will be stored in the columns "title", "id", and "text".

It is important to be aware of which datatype each of this columns is storing, else our algorithm will not function correctly.

In [0]:
# Count the total number of pages in the entire database, this will be the size of the dataset that we operate on
# N = wikipediaDF.count() 
# N will be the size of reverse df

In [0]:
# Instead of using the full database, we will use a smaller version to analyse the structure, with just 0.01% of records (approx. 582 records)
# We will select this small percentage of records randomly. In order to minimize random results, we will set a fixed seed value equal to 0

PartialWikipediaDF = wikipediaDF.sample(fraction=0.0001,seed=0).cache()

In [0]:
# Since this command is not relevant for our algorithm and takes a long time to run (7.88 min), we decided to comment it out. 
# If we do run it we can appreciate how much our dataset has diminished in comparison to the original one. 

# This command counts the number of pages in our new, reduced database
# PartialWikipediaDF.count()

In [0]:
# Display our smaller version of the database. This way we can see the information in an easier to read format. 
# For curiosity purposes, we also created a word cloud which displays the most frequent words used the largest. 

# As it is not necessary to view this table for the algorithm to work and it took a long time to run (7.59 min), we will be comenting this line out, as well.

# display(PartialWikipediaDF)

# 3. Extraction of relevant data
Now that we have a rough idea of our dataset and have decreased its size to a more manageable subset, we will begin with our analysis.

As we mentioned previously and highlighted when displayed as a table, the most important information for our algorithm is stored in the columns named “title”, “id”, and “text”. 
- Title stores the title of the document (string)
- ID stores the ID of the document (integer)
- Text stores the content of the document (string)

This is intuitive when thinking about the purpose of the page rank algorithm, which is to rank web pages in their search engine results. In order to complete this task, the algorithm will need to know the name (title) of the page, what the page is about (text), and the unique identity of the page to distinguish it from others (ID). The most important piece of information for this algorithm is are the links that the page content (text) includes, as the rating of importance each page will recieve is a recursively defined measure whereby a page becomes important if important pages link to it.

To create the table containing the forward links, we have to first identify which are the outgoing links (identified with double "[[]]") and then select the ID rows for these hyperlinks. Further, we will be ignoring all external references and resources, such as images. 

Thus, in this section we will focus on those three columns, and extract the key information from the text column, namely the links (identifiable as strings in brackets). To extract this information we will use regular expressions because regular expressions are a powerful tool to split texts into fragments.

As we will also be using Python functions that are not directly callable from the Spark Dataframe, we will be have to define some UDFs (user defined functions).

In [0]:
# Implementation of the parse_links function to parse the links in the document body
# This function recieves strings and returns returns all non-overlapping matches of the pattern in the string, as a list of strings. The string is scanned left-to-right, and matches are returned in the order found.

def parse_links(document_body):
  # document_body = document_body.lower() # Convert to lowercase. To optimize our code we chose to only convert to lowercase the pontential candidates that may be links instead of all the text 
  
  data = re.findall(r'\[\[(.+?)\]\]',document_body) # Locate the potential candidates that may be links
 
  if (len(data)>0): # Check if there are links in the article
    links = [] # Create empty list of links
    
    for s in data: # For loop to iterate through all candidate links
      s = s.lower() # Convert to lowercase 
      
    links = [s.split('|')[0].lower() for s in data]  # Get article name when [[name_article | text_in_article]]
        
    links = [s.split(':')[1] if "category:" in s else s for s in links]  # Get article name after a :
      
    links = [s for s in links if 'file' not in s]  # Ignore when [[File:...]] because this is not a relevant link for the page rank
        
    links = [s for s in links if 'image:' not in s] # Ignore when [[Image:...]] because this is not a relevant link for the page rank
        
    links = [s for s in links if 'video:' not in s] # Ignore when [[Video:...]] because this is not a relevant link for the page rank
    
    links = [s for s in links if 'template:' not in s] # Ignore when [[Template:...]] because this is not a relevant link for the page rank
         
    links = [s.split('#')[0] if "#" in s else s for s in links] # Ignore references or internal links within an article because these are not relevant links for the page rank
  
      
  else: # if there are no links in the article 
    links = []    # Create empty list of links
 
  return links


In [0]:
test="{{Use Indian English|date=April 2015}} {{Infobox person | name = Shavez Khan | image = | caption = | birth_date = | birth_place = India | nationality = India | residence = [[Mumbai]], India | occupation = [[Actor]] | years_active = present | height = }} '''Shavez Khan''' is an [[India]]n television [[actor]]. He has done his roles in various Indian television shows like Shaitaan,<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-feature-episodic-of-colors-shaitaan|title=Shavez Khan to feature in an episodic of Colors' Shaitaan|work=Tellychakkar|date=11 April 2013|accessdate=24 April 2015}}</ref> [[Encounter (Indian TV series)|Encounter]], [[Ek Hasina Thi (TV series)|Ek Hasina Thi]], [[Savdhaan India]],<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-anshul-singh-and-damini-joshi-episodic-of-savdhan-india-140915|title=Shavez Khan, Anshul Singh and Damini Joshi in an episodic of Savdhan India|work=Tellychakkar|date=15 September 2014|accessdate=24 April 2015}}</ref> [[SuperCops vs Supervillains]],<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/rituraj-singh-and-shavez-khan-life-oks-shapath-141009|title=Rituraj Singh and Shavez Khan in Life OK's Shapath|work=Tellychakkar|date=9 October 2014|accessdate=24 April 2015}}</ref> Pyaar Ka The End,<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-bindass-pyaar-ka-the-end-141029|title=Shavez Khan in Bindass' Pyaar Ka The End|work=Tellychakkar|date=29 October 2014|accessdate=24 April 2015}}</ref> [[Pyaar Kii Ye Ek Kahaani]], [[MTV Fanaah]], [[Crime Patrol (TV series)|Crime Patrol]]. He has played his recent role in [[Sony Entertainment Television (India)|Sony TV]]'s [[C.I.D. (Indian TV series)|CID]].<ref>{{cite web|url=http://www.tellychakkar.com/tv/tv-news/shavez-khan-sony-tvs-cid-150417|title=Shavez Khan in Sony TV's CID|work=Tellychakkar|date=17 April 2015|accessdate=24 April 2015}}</ref> ==Television== *[[Colors (TV channel)|Colors]]'s Shaitaan *[[Sony Entertainment Television (India)|Sony TV]]'s [[Encounter (Indian TV series)|Encounter]], [[Crime Patrol (TV series)|Crime Patrol]] & [[C.I.D. (Indian TV series)|CID]] *[[Star Plus]]'s [[Ek Hasina Thi (TV series)|Ek Hasina Thi]] *[[Life OK]]'s [[Savdhaan India]] & [[SuperCops vs Supervillains]] *[[Bindass]]' Pyaar Ka The End *[[Star One]]'s [[Pyaar Kii Ye Ek Kahaani]] *[[MTV]]'s [[MTV Fanaah]] ==References== {{Reflist}} ==External links== {{Persondata | NAME = Khan, Shavez | ALTERNATIVE NAMES = | SHORT DESCRIPTION = Indian model and television actor | DATE OF BIRTH = <!--Birth date has been contested. Do not add without providing a reliably published source with a reputation for editorial oversight--> | PLACE OF BIRTH = India | DATE OF DEATH = | PLACE OF DEATH = }} {{DEFAULTSORT:Khan, Shavez}} [[Category:Living people]] [[Category:Indian male television actors]] [[Category:Actors in Hindi television]] [[Category:Indian television personalities]]"

In [0]:
parse_links(test)

Out[11]: ['mumbai',
 'actor',
 'india',
 'actor',
 'encounter (indian tv series)',
 'ek hasina thi (tv series)',
 'savdhaan india',
 'supercops vs supervillains',
 'pyaar kii ye ek kahaani',
 'mtv fanaah',
 'crime patrol (tv series)',
 'sony entertainment television (india)',
 'c.i.d. (indian tv series)',
 'colors (tv channel)',
 'sony entertainment television (india)',
 'encounter (indian tv series)',
 'crime patrol (tv series)',
 'c.i.d. (indian tv series)',
 'star plus',
 'ek hasina thi (tv series)',
 'life ok',
 'savdhaan india',
 'supercops vs supervillains',
 'bindass',
 'star one',
 'pyaar kii ye ek kahaani',
 'mtv',
 'mtv fanaah',
 'living people',
 'indian male television actors',
 'actors in hindi television',
 'indian television personalities']

In [0]:
# User defined function to parse the links 
parse_links_udf = udf(parse_links,ArrayType(StringType()))

In [0]:
# User defined function to convert the text to lowercase (title and text)
tolower_udf = udf(lambda x: x.lower())

In [0]:
# Create a dataframe with the selected information and renaming title to title and text to links
dataDF1 = PartialWikipediaDF.select(tolower_udf("title").alias("title"),"id",parse_links_udf("text").alias("links"))

In [0]:
display(dataDF1)

title id links godzilla 11986 List(godzilla (franchise), godzilla (1954 film), godzilla (1954 film), godzilla (2014 film), tomoyuki tanaka, ishirō honda, eiji tsubaraya, haruo nakajima, takeo murata, ishirō honda, godzilla (1954 film), dvd, classic media, al c. ward, ishirō honda, terry morse, godzilla, king of the monsters!, dvd, classic media, katsumi tezuka, yū sekida, seiji onaka, toru kawai, kenpachiro satsuma, tsutomu kitagawa, mizuho yoshida (actor), akira watanabe (film art director), teizô toshimitsu, godzilla, king of the monsters!, ishirō honda, terry o. morse, toho, godzilla vs. megaguirus, toho, godzilla: final wars, ryuhei kitamura, toho, godzilla raids again, toho, invasion of astro-monster, ishirō honda, toho, shusuke kaneko, toho, godzilla (2014 film), gareth edwards (director), legendary pictures, kaiju, kaiju, tokusatsu, godzilla (franchise), japan, ishirō honda, godzilla (1954 film), godzilla video games, godzilla (comics), godzilla (franchise), toho, godzilla_(franchise), godzilla, king of the monsters!, nuclear bombings of hiroshima and nagasaki, daigo fukuryū maru, nuclear weapon, portmanteau, gorilla, whale, ateji, kanji, hepburn romanization, kunrei romanization, ray harryhausen, the beast from 20,000 fathoms, amphibian, dinosaur, wired (magazine), tyrannosaurus, iguanodon, stegosaurus, alligator, usa today, chimera (mythology), life (magazine), keloid, hibakusha, turner classic movies, akira ifukube, double bass, godzilla 2000, black belt (martial arts), godzilla raids again, transitional form, regeneration (biology), godzilla 2000, takao okawara, toho, godzilla vs. king ghidorah, kazuki ōmori, toho, godzilla vs. mechagodzilla, jun fukuda, toho, godzilla vs. biollante, kazuki ōmori, toho, godzilla: destroy all monsters melee, pipeworks software, zone fighter, the godzilla power hour, godzilla vs. hedorah, yoshimitsu banno, toho, ghidorah, the three-headed monster, ishirō honda, toho, godzilla: unleashed, the return of godzilla, koji hashimoto (director), toho, shinto, king kong vs. godzilla, gender-neutral pronoun, godzilla (1998 film), parthogenesis, jsdf, king ghidorah, gigan, mechagodzilla, megalon, biollante, megaguirus, mothra, rodan, anguirus, minilla, king kong, fantastic four, godzilla (1954 film), godzilla, king of the monsters!, joseph e. levine, tokyo metropolitan government building, godzilla vs. king ghidorah, legendary pictures, godzilla (2014 film), gareth edwards (director), godzilla (2016 film), suitmation, animatronics, stop-motion, computer generated imagery, king kong (1933 film), eiji tsuburaya, the return of godzilla, godzilla vs biollante, godzilla vs destoroyah, kenpachiro satsuma, oxygen deprivation, godzilla vs. spacegodzilla, patrick tatopoulos, motion capture, digitigrade, computer-generated imagery, stan winston, jurassic park (film), the moving picture company, bear, komodo dragon, lizard, lion, gray wolf, the hollywood reporter, popular culture, kaijū, tokusatsu, metaphor, atomic bombings of hiroshima and nagasaki, mtv movie awards, hollywood walk of fame, godzilla: final wars, bambi meets godzilla, mystery science theater 3000, godzilla (song), blue öyster cult, nike, inc., basketball, charles barkley, godzilla vs. charles barkley, jeff butler, gamera, yonggary, gorgo (film), gojirasaurus, nomen dubium, coelophysidae, paleontologist, kenneth carpenter, dakosaurus, thalattosuchia, jurassic, nickname, ceratosauria, darren naish, public domain, generic trademark, subway (restaurant), honda, honda odyssey, sea shepherd conservation society, toho, gojira (band), technical death metal, voltage pictures, anne hathaway, shinjuku, toho, godzilla characters, toho monsters, fictional characters introduced in 1954, horror film characters, fictional mutants, fictional dinosaurs, fictional characters with superhuman strength, science fiction film characters, king kong characters, fictional characters with nuclear or radiation abilities, fictional telepaths) george eliot 13049 List(nu

# 4. Creation of a table of forward links
As our goal is to know how many links each document forwards to, we will now transform our list of links to a list of document IDs and create a table with them. In order to extract the ID of the target linked documents (the pages that are linked) we will have to analyse the entire Wikipedia database and create a table with these two pieces of information. 

The table of forward links with feature a column of page IDs and another column with containing all the target pages of that ID ( "origin" page).

In [0]:
# Extract the data (ID and title) from the full Wikipedia DF & store it in a new Spark DF
titleidDF = wikipediaDF.select("id",lower("title").alias("title"))

In [0]:
# Convert from Spark DF to a Pandas DF that contains the title and ID of the documents
titleidPDF = titleidDF.toPandas()

In [0]:
# Extract the data from the partial Wikipedia DF & store it in a new Spark DF
TempForwardDF = PartialWikipediaDF.select("title","id",parse_links_udf("text").alias("links"))

In [0]:
display(TempForwardDF)

title id links Godzilla 11986 List(godzilla (franchise), godzilla (1954 film), godzilla (1954 film), godzilla (2014 film), tomoyuki tanaka, ishirō honda, eiji tsubaraya, haruo nakajima, takeo murata, ishirō honda, godzilla (1954 film), dvd, classic media, al c. ward, ishirō honda, terry morse, godzilla, king of the monsters!, dvd, classic media, katsumi tezuka, yū sekida, seiji onaka, toru kawai, kenpachiro satsuma, tsutomu kitagawa, mizuho yoshida (actor), akira watanabe (film art director), teizô toshimitsu, godzilla, king of the monsters!, ishirō honda, terry o. morse, toho, godzilla vs. megaguirus, toho, godzilla: final wars, ryuhei kitamura, toho, godzilla raids again, toho, invasion of astro-monster, ishirō honda, toho, shusuke kaneko, toho, godzilla (2014 film), gareth edwards (director), legendary pictures, kaiju, kaiju, tokusatsu, godzilla (franchise), japan, ishirō honda, godzilla (1954 film), godzilla video games, godzilla (comics), godzilla (franchise), toho, godzilla_(franchise), godzilla, king of the monsters!, nuclear bombings of hiroshima and nagasaki, daigo fukuryū maru, nuclear weapon, portmanteau, gorilla, whale, ateji, kanji, hepburn romanization, kunrei romanization, ray harryhausen, the beast from 20,000 fathoms, amphibian, dinosaur, wired (magazine), tyrannosaurus, iguanodon, stegosaurus, alligator, usa today, chimera (mythology), life (magazine), keloid, hibakusha, turner classic movies, akira ifukube, double bass, godzilla 2000, black belt (martial arts), godzilla raids again, transitional form, regeneration (biology), godzilla 2000, takao okawara, toho, godzilla vs. king ghidorah, kazuki ōmori, toho, godzilla vs. mechagodzilla, jun fukuda, toho, godzilla vs. biollante, kazuki ōmori, toho, godzilla: destroy all monsters melee, pipeworks software, zone fighter, the godzilla power hour, godzilla vs. hedorah, yoshimitsu banno, toho, ghidorah, the three-headed monster, ishirō honda, toho, godzilla: unleashed, the return of godzilla, koji hashimoto (director), toho, shinto, king kong vs. godzilla, gender-neutral pronoun, godzilla (1998 film), parthogenesis, jsdf, king ghidorah, gigan, mechagodzilla, megalon, biollante, megaguirus, mothra, rodan, anguirus, minilla, king kong, fantastic four, godzilla (1954 film), godzilla, king of the monsters!, joseph e. levine, tokyo metropolitan government building, godzilla vs. king ghidorah, legendary pictures, godzilla (2014 film), gareth edwards (director), godzilla (2016 film), suitmation, animatronics, stop-motion, computer generated imagery, king kong (1933 film), eiji tsuburaya, the return of godzilla, godzilla vs biollante, godzilla vs destoroyah, kenpachiro satsuma, oxygen deprivation, godzilla vs. spacegodzilla, patrick tatopoulos, motion capture, digitigrade, computer-generated imagery, stan winston, jurassic park (film), the moving picture company, bear, komodo dragon, lizard, lion, gray wolf, the hollywood reporter, popular culture, kaijū, tokusatsu, metaphor, atomic bombings of hiroshima and nagasaki, mtv movie awards, hollywood walk of fame, godzilla: final wars, bambi meets godzilla, mystery science theater 3000, godzilla (song), blue öyster cult, nike, inc., basketball, charles barkley, godzilla vs. charles barkley, jeff butler, gamera, yonggary, gorgo (film), gojirasaurus, nomen dubium, coelophysidae, paleontologist, kenneth carpenter, dakosaurus, thalattosuchia, jurassic, nickname, ceratosauria, darren naish, public domain, generic trademark, subway (restaurant), honda, honda odyssey, sea shepherd conservation society, toho, gojira (band), technical death metal, voltage pictures, anne hathaway, shinjuku, toho, godzilla characters, toho monsters, fictional characters introduced in 1954, horror film characters, fictional mutants, fictional dinosaurs, fictional characters with superhuman strength, science fiction film characters, king kong characters, fictional characters with nuclear or radiation abilities, fictional telepaths) George Eliot 13049 List(nu

In [0]:
# Extract the ID value of the links in the title column 

def titles2id(links, titleidPDF):
  data_titles = titleidPDF
  
  if (len(links)>0): # Check if there are links
    ids = data_titles[data_titles.title.isin(links)].id.to_list() # Return a list of IDs from data_titles that match the titles in links
    
  else: # If there are no links
    ids = [] # Create empty list 
    
  return list(set(ids)) # Convert the result to a list 

In [0]:
# Create a UDF function of the titles2id function 

titles2id_UDF = udf(lambda x: titles2id(x,titleidPDF),ArrayType(LongType(),False))

# we create a user define function of the text_links_2idx function
#titles2id_UDF = udf(titles2id, ArrayType(LongType()))

In [0]:
# Extract the IDs of all the links a page forwards to in order to create the forward links table 
ForwardDF = TempForwardDF.select("id",titles2id_UDF("links").alias("links")).cache()

In [0]:
# Display the IDs of all the links a page forwards to, i.e. the forward links table
display(ForwardDF)

id,links
11986,"List(36865, 438275, 31626763, 303628, 79887, 1190931, 36896, 930853, 20518, 1670693, 925736, 3135027, 1222711, 854081, 567874, 538696, 208463, 634965, 187479, 1922658, 18653283, 18637926, 1947751, 267370, 17229419, 621, 28272, 4627056, 8311, 1236610, 36101762, 1862287, 21123733, 1009817, 396968, 1236650, 30313654, 53430, 4259513, 302275, 4726468, 870601, 2348754, 11988, 15573, 5220058, 325852, 12000, 12004, 12005, 37604, 45178603, 9073903, 325875, 3572981, 9182971, 1909502, 9322751, 12546, 30467, 18993927, 18184, 226575, 490771, 192790, 147734, 1646358, 21785, 7396632, 23188764, 18057502, 1112869, 15655, 2120494, 4400, 105776, 54581, 3921, 162143, 11014498, 894309, 3540339, 1245556, 791422, 23624578, 65411, 11778948, 68485, 11664, 8660880, 570258, 570263, 1356696, 1252253, 1790878, 13729, 18637733, 33702, 570279, 16823212, 1407410, 248760, 228798, 30863297, 2156483, 17360, 30683, 266204, 20002271, 59361, 5765607, 65001, 9182702, 33777, 18998781)"
13049,"List(42368, 230019, 6532, 33925, 228998, 2427653, 29604227, 21244047, 94097, 12785172, 34543638, 199961, 21018, 323481, 208411, 248859, 32798, 670749, 383648, 294177, 15782, 18622119, 2799016, 3038250, 70188, 8495660, 213936, 4368178, 47923, 147635, 23953851, 700, 875196, 27480261, 1550022, 87240, 1972169, 32915915, 189774, 153295, 45777, 7997906, 22928212, 26202, 213978, 1047259, 803804, 44766, 310113, 946273, 209892, 39658852, 32742, 12521, 1129578, 5178089, 10891114, 43245, 1056621, 6148078, 16320493, 1128563, 19444, 34115577, 2922874, 161275, 3745788, 26255613)"
38010,"List(31489, 43638795, 18957, 11024, 30206738, 22303, 460321, 3117601, 18994087, 291242, 49838, 19334830, 23868856, 13256, 42480973, 459471, 38481, 200033, 9955, 44905, 294126, 18209535)"
38252,"List(43008, 57858, 259082, 797714, 48147, 42010, 156699, 2000410, 156702, 2943008, 10277, 494630, 5166, 2776113, 14733874, 51763, 22068, 41523, 72758, 2555445, 174650, 2416701, 21611071, 2465857, 273476, 431685, 198217, 1926730, 186444, 273484, 273486, 273487, 431695, 273489, 2011729, 18949200, 30866008, 44635, 6126173, 11690592, 22113, 1072228, 4883556, 152171, 3823213, 191086, 915566, 18933360, 43956336, 29812, 13696118, 21113, 238715, 1102461, 9282173, 39483010, 19344515, 31365, 25063047, 43144329, 42634, 8803981, 491664, 148113, 1775761, 28309, 371350, 13696153, 2416795, 23648411, 303261, 32927, 146595, 24007843, 22944933, 66730, 79026, 87731, 1721523, 25804468, 462002, 25817785, 228538, 171195, 18994363, 437949, 182462, 5369534, 16579, 591556, 3852997, 232142, 1407696, 639185, 92374, 1574616, 24061144, 31514330, 31453, 2530527, 730350, 21231, 2004720, 30456, 423161, 30419193, 587003, 5126908, 15613, 1588483, 19698439, 32009, 10510, 19344654, 53518, 170771, 170772, 170773, 205078, 511253, 954140, 2041117, 1170208, 22308, 7129894, 9510, 47777069, 355118, 51503, 205105, 81213, 6974, 172350, 28480, 1732421, 29511, 6766412, 13825869, 18309966, 2798927, 140624, 140630, 1462102, 238424, 155482, 7038301, 34041182, 27803997, 719715, 20836, 52581, 235879, 38252, 27705200, 136566, 1029495, 1654648, 170874, 77179, 4477, 1121666, 65411, 156547, 180103, 17292, 1529230, 37610899, 2186651, 202652, 20110751, 1712034, 7444902, 486312, 30860202, 45294511, 416688, 17339824, 2632114, 12610483, 20405, 276919, 16728504, 1988033, 612293, 1560005, 2291656, 4865481, 8642505, 3897803, 2520013, 6062032, 319442, 5079506, 53207, 28178906, 928738, 1714150, 24047593, 13603306, 27424748, 805870, 921071, 167409, 1571314, 1040371, 7668, 32722929, 33642486, 13695994, 2808829, 12799)"
196900,"List(198400, 51847, 51848, 73100, 161935, 3985, 50585, 68145, 199608, 324, 44748, 26956, 180432, 198354, 42524885, 201174, 6547798, 140632, 197469, 325863, 50287, 7672)"
201227,"List(333058, 31756, 24850, 21780, 50714, 396572, 27040, 24909346, 26787, 14705955, 73257, 146730, 501161, 11054, 102446, 24113, 2320314, 8569916, 32190, 2731583, 400064, 21347657, 31770697, 2764750, 55779, 30439, 40558, 382576, 765430, 350712, 3434750)"
324799,"List(29628

# 5. Computation of the number of outgoing links
Now that we have created the forward links matrix, we will compute the backward links matrix. For this we will compute the number of outgoing links for each page (document) using our previous forward links DF. This is a crucial step because our page rank algorithm needs this information in order to adequately perfom its ranking and importance calculations.

To perform this computation we will use the PySpark function "size" to check the length of the links in the ForwardDF dataframe.

In [0]:
# Compute the number of outgoing links for each document
OutgoingsLinksCountersDF = ForwardDF.select("id",size("links").alias("counter"))

In [0]:
# Convert from Spark DF to a Pandas DF
OutgoingsLinksCountersDFPDF = OutgoingsLinksCountersDF.toPandas()

In [0]:
display(OutgoingsLinksCountersDF)

id,counter
11986,115
13049,68
38010,22
38252,211
196900,22
201227,31
324799,13
162369,13
340927,27
343031,50


# 6. Creation of a table of reverse links 
Next, we will create the table of reverse links using the previously computed forward links DF. This step is also important, as our page rank algorithm needs to know which links are directed to each document.

In [0]:
# Create a new dataframe (TemporalReverseLinks) from an existing one (ForwardDF)
TemporalReverseLinks = ForwardDF.select("id",explode("links").alias("t_link"))

In [0]:
# Display the first 10 rows
TemporalReverseLinks.show(10)

+-----+--------+
|   id|  t_link|
+-----+--------+
|11986|   36865|
|11986|  438275|
|11986|31626763|
|11986|  303628|
|11986|   79887|
|11986| 1190931|
|11986|   36896|
|11986|  930853|
|11986|   20518|
|11986| 1670693|
+-----+--------+
only showing top 10 rows



In [0]:
# Create a new DF (ReverseDF) by grouping the TemporalReverseLinks DF by the t_link column and aggregating the id column using collect_list
# The result is cached in memory

ReverseDF = TemporalReverseLinks.groupBy("t_link").agg(collect_list ("id").alias("links")).cache()

In [0]:
display(ReverseDF)

t_link,links
3572981,List(11986)
209892,List(13049)
50287,List(196900)
300208,List(729744)
323434,List(1435291)
15237,List(2542615)
2164250,List(2400112)
445232,List(3164637)
18582511,List(3418750)
92357,List(7930852)


In [0]:
# # option 2. variables are not correct for our case!!! check if we should alter our previous code (which is mostly the teacher's)

# # use the ForwardDF dataframe and python that will reverse the ForwardDF table, by putting as an id the ids in array of the links column, and putting in another column the id that it is coming from. Then we groupBy the ids, and create a dataFrame that will contain an id column that refers to a document and a links column that contains an array with all the documents that contain as a link the document corresponding to the id article.

# # Reverse Table
# def reverseId(id,links):
#     if (len(links)>0):
#         reverse = list(set([ (tgt_id,id) for tgt_id in links]))
#     else:
#         reverse=[]
#     return reverse

# ForwardRDD = ForwardDF.rdd

# #flatMap the reverseID function, and then groupBy the ids. 
# ReverseRDD = (ForwardRDD
#             .flatMap(lambda r: reverseId(r.id, r.links))
#             .groupByKey()
#             .map(lambda r: (r[0], list(r[1]))))

# ReverseDF = spark.createDataFrame(ReverseRDD, ["id", "links"])

# display(ReverseDF)


# #optional 
# # then, To improve the Page Rank calculation, add an extra column, that contains and array with the number of output links that each page in the links column contains. Which means that this extra column will contain an array of the same length as the array in the links column.
# # To do this use the count_linksPDF pandas dataframe and a python function that will look in the count_linksPDF the number of outputs for each id in the "links" column.

# # broadcast the count_linksPDF variable to be able to use it in the lambda mapping inside of the ReverseDF dataframe. 
# broadcast_output_number = sc.broadcast(count_linksPDF)

# def links_to_num_outputs(links, count_links):
#     result = count_links[count_links.id.isin(links)].num_outputs.to_list()
    
#     return result

# # convert the links_to_num_outputs function to a user define function variable
# udf_links_2_num_outputs = udf(lambda l: links_to_num_outputs(l, count_linksPDF), ArrayType(LongType()))

# # map the function to each row to create the new column
# ReverseDF = ReverseDF.select("Id", "links", udf_links_2_num_outputs("links").alias("num_outputs_links")).cache()

# display(ReverseDF)

# 7. Creation of the page rank table 
Now that we have computed the forward, backward, and reverse links matrices, we will use the algorithm described in Brin and Page's paper to set the PageRank for each page. Our goal is to create a table that displays the page rank by ID. 

We will initialise the rank number to be 0,85 / N, where N is the total number of pages in the full Wikipedia database and 0,85 is the so-called damping factor, i.e. the click-through probability (probability that a user will go to the linked web page).

The pagerankDF will include the same IDs are the ones in the Reverse DF.

In [0]:
# Create a Pandas DF from the reverseDF
pageRankPDF = ReverseDF.select("t_link").toPandas()

In [0]:
# Compute the page rank with the formula explained above
N = ReverseDF.count() 
pageRankPDF["PageRank"] = 0.85/N #N is the total number of documents in the full Wikipedia database. We already computed this value at the beginning

In [0]:
# Display the page rank table 

display(pageRankPDF)

t_link,PageRank
3572981,1.1324273914201971E-4
209892,1.1324273914201971E-4
50287,1.1324273914201971E-4
300208,1.1324273914201971E-4
323434,1.1324273914201971E-4
15237,1.1324273914201971E-4
2164250,1.1324273914201971E-4
445232,1.1324273914201971E-4
18582511,1.1324273914201971E-4
92357,1.1324273914201971E-4


# 8. Definition of the number of iterations for the page rank
Now we need to determine how many times the page rank should calculate the rank. To do this, we will use a loop. 
We will set a maximum of 20 iterations to calculate the pagerank. In the case that our algorithm were not to converge in this number of iterations, the loop will stop and display the end results.

In [0]:
# broadcast_count_linksPDF = sc.broadcast(OutgoingsLinksCountersDFPDF)

def new_pagerank(link_id, pr_dataframe, links):
  n_pr = 0
#   count_links = broadcast_count_linksPDF.value
  print("dupa")
  for l in links:
    # get current page rank
    current_pr = pr_dataframe[pr_dataframe["t_link"] == l]
#     print(current_pr)
    if len(current_pr)>0:                               # check if the page exists in pr_dataframe
      current_pr = current_pr.iloc[0][1]                # In the case that we have some pages that links with our current_id, the PR value is the one that is defined in our current_page_rank
    else:                                                     
      current_pr = 0
    # get page outgoing links number
    outgoing = OutgoingsLinksCountersDFPDF[OutgoingsLinksCountersDFPDF["id"] == i]     # The for the count value we will follow the same idea, we recover from the count_links the count number
    if len(outgoing)>0:                                           # We check if this page has links
      outgoing = outgoing.iloc[0][1]                                 # We store this value in count variable
      n_pr += current_pr/outgoing
  new_pr = 0.85/N+0.15*n_pr
  return new_pr

In [0]:
new_pagerank_udf = udf (lambda x,y: new_pagerank(x,pageRankPDF,y), DoubleType())

In [0]:
# WHILE LOOP WITH SEVERAL EXIT CONDITIONS
i = 0
while i<2:
    new_pagerank_udf = udf (lambda x,y: new_pagerank(x,pageRankPDF,y), DoubleType())
    NewPageRankDF=ReverseDF.select(ReverseDF["t_link"],new_pagerank_udf(ReverseDF["t_link"],ReverseDF["links"]).alias("PageRank"))
    pageRankPDF=NewPageRankDF.toPandas()
    print(i)
    i+=1

0
1


In [0]:
display(pageRankPDF)

t_link,PageRank
3572981,1.1324273914201971E-4
209892,1.1324273914201971E-4
50287,1.1324273914201971E-4
300208,1.1324273914201971E-4
323434,1.1324273914201971E-4
15237,1.1324273914201971E-4
2164250,1.1324273914201971E-4
445232,1.1324273914201971E-4
18582511,1.1324273914201971E-4
92357,1.1324273914201971E-4


# Conclusion